In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [3]:
def univariate(data, look_back):
    signal_data =data
    # 데이터 전처리
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal_data = scaler.fit_transform(signal_data)

    # 데이터 분리
    train = signal_data[0:120]
    #val = signal_data[800:1200]
    test = signal_data[120:]

    # 데이터셋 생성
    x_train, y_train = create_dataset(train, look_back)
    #x_val, y_val = create_dataset(val, look_back)
    x_test, y_test = create_dataset(test, look_back)

    x_train = np.reshape(x_train,(x_train.shape[0],look_back,1)) #(size, timestamp,feature)
    x_test = np.reshape(x_test,(x_test.shape[0],look_back,1))
    
    return x_train, x_test, y_train, y_test,scaler, look_back

In [4]:
def run_bi(x_train, x_test, y_train, y_test,look_back, scaler,box):
    

    model = Sequential()
    model.add(Bidirectional(LSTM(box, input_shape=(look_back, 1)))) #(timestep,feature)
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

    model.fit(x_train, y_train, epochs=100, batch_size=5, verbose=1, callbacks=[early_stop])
    score = model.evaluate(x_test,y_test,batch_size=5)
    pre =model.predict(x_test)
    model.reset_states()
    #sc_pre = scaler.inverse_transform(pre)
    ans = y_test
    
    return score, pre, ans

In [5]:
def run_stateful(x_train, x_test, y_train, y_test, look_back, scaler,box):
    model = Sequential()
    model.add(LSTM(box, batch_input_shape=(1, look_back, 1), stateful=True))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

    model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop])
    score = model.evaluate(x_test,y_test,batch_size=1)
    pre =model.predict(x_test,batch_size=1) #모든 batch_size 바꾸면 error
    #sc_pre = scaler.inverse_transform(pre)
    ans = y_test
    
    return score, pre, ans

In [6]:
def run_stack(x_train, x_test, y_train, y_test,look_back, scaler,box,box2):

    model = Sequential()
    model.add(LSTM(box, input_shape=(look_back, 1), return_sequences=True))#(timestep,feature)
    model.add(LSTM(box2))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

    model.fit(x_train, y_train, epochs=100, batch_size=5, verbose=1, callbacks=[early_stop])
    score = model.evaluate(x_test,y_test,batch_size=5)
    pre =model.predict(x_test)
    #sc_pre = scaler.inverse_transform(pre)
    ans = y_test
    
    return score, pre, ans

In [7]:
def run_stateful_stack(x_train, x_test, y_train, y_test, look_back, scaler,box1,box2):
    model = Sequential()
    model.add(LSTM(box1, batch_input_shape=(1, look_back, 1), stateful=True,return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(box2, batch_input_shape=(1, look_back, 1), stateful=True))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

    model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop])
    score = model.evaluate(x_test,y_test,batch_size=1)
    pre =model.predict(x_test,batch_size=1) #모든 batch_size 바꾸면 error
    #sc_pre = scaler.inverse_transform(pre)
    ans = y_test
    
    return score, pre, ans

In [8]:
PATH ='C:/Users/ckdrj/Google 드라이브/baseball/data/total/'
#PATH = 'C:/Users/LIM CHANG GEON/Google 드라이브/baseball/data/total'
hiter16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv")
pitcher16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv")

### usage
univariate(data_feature,look_back)
run_function select (~,box) or (~,box1,box2)

In [9]:
x_train, x_test, y_train, y_test,scaler,look_back = univariate(pitcher16[["R"]],10)
score, pre, ans = run_stateful(x_train, x_test, y_train, y_test,look_back, scaler,10)
#score, pre, ans = run_bi(x_train, x_test, y_train, y_test,look_back, scaler,10)
#score, pre, ans = run_stack(x_train, x_test, y_train, y_test,look_back, scaler,10,10)
score, pre, ans = run_stateful_stack(x_train, x_test, y_train, y_test,look_back, scaler,10,10)
sc_pre=scaler.inverse_transform(pre)
sc_ans=scaler.inverse_transform(ans.reshape(ans.shape[0],1))
print(score, sc_pre,sc_ans)

W0919 15:07:33.295110  3260 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 15:07:33.315114  3260 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 15:07:33.317114  3260 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 15:07:33.456131  3260 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0919 15:07:33.462110  3260 deprecation.py:506] From F:\sda\envs\d

Epoch 1/100
110/110 [==============================] - 1s 9ms/step - loss: 0.0517
Epoch 2/100
110/110 [==============================] - 1s 5ms/step - loss: 0.0319
Epoch 3/100
110/110 [==============================] - 1s 5ms/step - loss: 0.0343
Epoch 4/100
110/110 [==============================] - 1s 5ms/step - loss: 0.0317
Epoch 5/100
110/110 [==============================] - 1s 5ms/step - loss: 0.0357
Epoch 6/100
110/110 [==============================] - 1s 5ms/step - loss: 0.0343
Epoch 7/100
110/110 [==============================] - 1s 5ms/step - loss: 0.0348
Epoch 00007: early stopping
1310/1310 [==============================] - 2s 1ms/step
Epoch 1/100
110/110 [==============================] - 2s 19ms/step - loss: 0.0401
Epoch 2/100
110/110 [==============================] - 1s 8ms/step - loss: 0.0379
Epoch 3/100
110/110 [==============================] - 1s 8ms/step - loss: 0.0330
Epoch 4/100
110/110 [==============================] - 1s 8ms/step - loss: 0.0351
Epoch 5/100


In [9]:
loss = 0
team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']
loss_min=1000
for lookback in [3,5,10]:
    for box in [5, 10, 15]:
        loss = 0
        for team in team_list:
            print(lookback, box, team)
            pitcher=pitcher16[pitcher16.T_ID==team]
            x_train, x_test, y_train, y_test,scaler,look_back = univariate(pitcher[["R"]],lookback)
            #score, pre, ans = run_stateful(x_train, x_test, y_train, y_test,look_back, scaler,box)
            score, pre, ans = run_bi(x_train, x_test, y_train, y_test,look_back, scaler,box)
            #score, pre, ans = run_stack(x_train, x_test, y_train, y_test,look_back, scaler,box,box)
            #score, pre, ans = run_stateful_stack(x_train, x_test, y_train, y_test,look_back, scaler,box,box)
            sc_pre=scaler.inverse_transform(pre)
            sc_ans=scaler.inverse_transform(ans.reshape(ans.shape[0],1))
            print(score, sc_pre,sc_ans)
            loss += score
            del pitcher
        print(loss)
        if loss <loss_min:
            loss_min=loss
            box_min=box
            look_back_min=look_back
        
print(loss_min, box_min,look_back_min)


W0920 00:10:50.852432 16044 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0920 00:10:50.866417 16044 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0920 00:10:50.880418 16044 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0920 00:10:50.882416 16044 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



3 5 WO


W0920 00:10:51.173418 16044 deprecation_wrapper.py:119] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0920 00:10:51.179415 16044 deprecation.py:506] From F:\sda\envs\dlstudy\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0920 00:10:51.486431 16044 deprecation.py:323] From F:\sda\envs\dlstudy\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0920 00:10:52.228432

Epoch 1/100
117/117 [==============================] - 1s 10ms/step - loss: 0.1071
Epoch 2/100
117/117 [==============================] - 0s 650us/step - loss: 0.0704
Epoch 3/100
117/117 [==============================] - 0s 624us/step - loss: 0.0541
Epoch 4/100
117/117 [==============================] - 0s 624us/step - loss: 0.0442
Epoch 5/100
117/117 [==============================] - 0s 684us/step - loss: 0.0428
Epoch 6/100
117/117 [==============================] - 0s 692us/step - loss: 0.0423
Epoch 7/100
117/117 [==============================] - 0s 778us/step - loss: 0.0404
Epoch 8/100
117/117 [==============================] - 0s 761us/step - loss: 0.0424
Epoch 9/100
117/117 [==============================] - 0s 709us/step - loss: 0.0404
Epoch 10/100
117/117 [==============================] - 0s 761us/step - loss: 0.0408
Epoch 00010: early stopping
21/21 [==============================] - 0s 8ms/step
0.0686357519811108 [[5.5091476]
 [5.4952846]
 [5.9650946]
 [5.876466 ]
 [5.0878

Epoch 1/100
117/117 [==============================] - 2s 20ms/step - loss: 0.0808
Epoch 2/100
117/117 [==============================] - 0s 684us/step - loss: 0.0605
Epoch 3/100
117/117 [==============================] - 0s 692us/step - loss: 0.0534
Epoch 4/100
117/117 [==============================] - 0s 676us/step - loss: 0.0513
Epoch 5/100
117/117 [==============================] - 0s 726us/step - loss: 0.0523
Epoch 6/100
117/117 [==============================] - 0s 735us/step - loss: 0.0516
Epoch 7/100
117/117 [==============================] - 0s 744us/step - loss: 0.0476
Epoch 8/100
117/117 [==============================] - 0s 735us/step - loss: 0.0517
Epoch 9/100
117/117 [==============================] - 0s 744us/step - loss: 0.0506
Epoch 10/100
117/117 [==============================] - 0s 752us/step - loss: 0.0516
Epoch 00010: early stopping
21/21 [==============================] - 1s 32ms/step
0.02653786135524478 [[5.6867576]
 [5.644404 ]
 [5.961869 ]
 [6.7382226]
 [5.68

...

Epoch 1/100
115/115 [==============================] - 9s 78ms/step - loss: 0.0596
Epoch 2/100
115/115 [==============================] - 0s 1ms/step - loss: 0.0446
Epoch 3/100
115/115 [==============================] - 0s 983us/step - loss: 0.0455
Epoch 4/100
115/115 [==============================] - 0s 983us/step - loss: 0.0452
Epoch 5/100
115/115 [==============================] - 0s 1ms/step - loss: 0.0460A: 0s - loss: 0.03
Epoch 00005: early stopping
19/19 [==============================] - 3s 158ms/step
0.04607305352232958 [[4.9042244]
 [4.7145386]
 [5.192309 ]
 [5.5381846]
 [4.787344 ]
 [4.6347885]
 [4.789135 ]
 [4.009462 ]
 [3.138021 ]
 [3.2591178]
 [3.1567059]
 [3.65226  ]
 [3.574091 ]
 [3.6465597]
 [3.9413383]
 [4.212486 ]
 [4.6106606]
 [5.6555333]
 [5.835605 ]] [[ 2.]
 [10.]
 [ 9.]
 [ 2.]
 [ 2.]
 [ 0.]
 [ 3.]
 [ 3.]
 [ 4.]
 [ 1.]
 [ 4.]
 [ 2.]
 [ 4.]
 [ 9.]
 [ 3.]
 [ 9.]
 [13.]
 [ 5.]
 [ 1.]]
5 5 NC
Epoch 1/100
115/115 [==============================] - 9s 82ms/step - loss:

115/115 [==============================] - 0s 1ms/step - loss: 0.0505
Epoch 10/100
115/115 [==============================] - 0s 1ms/step - loss: 0.0539
Epoch 11/100
115/115 [==============================] - 0s 1ms/step - loss: 0.0519
Epoch 12/100
115/115 [==============================] - 0s 1ms/step - loss: 0.0536
Epoch 00012: early stopping
19/19 [==============================] - 6s 292ms/step
0.08074918193252463 [[5.803973 ]
 [6.034011 ]
 [5.6380076]
 [5.339609 ]
 [4.708072 ]
 [4.126685 ]
 [4.750453 ]
 [5.304998 ]
 [5.4351225]
 [5.7023416]
 [6.5810256]
 [6.2271733]
 [6.0041842]
 [6.5603724]
 [6.4898973]
 [6.2233944]
 [5.330268 ]
 [5.2538977]
 [4.25325  ]] [[ 4.]
 [ 0.]
 [ 3.]
 [ 3.]
 [ 1.]
 [14.]
 [ 7.]
 [ 5.]
 [ 6.]
 [ 9.]
 [13.]
 [ 4.]
 [13.]
 [ 2.]
 [ 4.]
 [ 3.]
 [ 0.]
 [ 4.]
 [ 6.]]
5 10 LG
Epoch 1/100
115/115 [==============================] - 14s 122ms/step - loss: 0.1344
Epoch 2/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0731
Epoch 3/100
115/115 [

Epoch 1/100
115/115 [==============================] - 20s 177ms/step - loss: 0.0938
Epoch 2/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0435
Epoch 3/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0474
Epoch 4/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0437
Epoch 5/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0428
Epoch 6/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0423
Epoch 7/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0438
Epoch 8/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0433
Epoch 9/100
115/115 [==============================] - 0s 2ms/step - loss: 0.0434
Epoch 00009: early stopping
19/19 [==============================] - 7s 365ms/step
0.031045389802832352 [[5.6167855]
 [5.884362 ]
 [6.142521 ]
 [6.0694957]
 [5.7297816]
 [5.806825 ]
 [5.6871705]
 [5.490153 ]
 [5.4309683]
 [5.3848443]
 [5.2226067]
 [5.1095514]

110/110 [==============================] - 27s 248ms/step - loss: 0.0831
Epoch 2/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0437
Epoch 3/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0437
Epoch 4/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0425
Epoch 5/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0442
Epoch 6/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0415
Epoch 7/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0425
Epoch 8/100
110/110 [==============================] - ETA: 0s - loss: 0.042 - 0s 3ms/step - loss: 0.0428
Epoch 9/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0439
Epoch 00009: early stopping
14/14 [==============================] - 10s 700ms/step
0.046329936278717856 [[4.852383 ]
 [4.717092 ]
 [4.6701946]
 [4.6576405]
 [4.4495826]
 [4.3708053]
 [4.52815  ]
 [4.185112 ]
 [3.8659315]
 [4.066424 ]
 [4.124494 ]

110/110 [==============================] - 0s 3ms/step - loss: 0.0561
Epoch 5/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0549
Epoch 6/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0574
Epoch 00006: early stopping
14/14 [==============================] - 12s 875ms/step
0.042241386004856656 [[6.4998174]
 [6.928073 ]
 [7.4403234]
 [7.0694985]
 [7.1808953]
 [6.630455 ]
 [6.514669 ]
 [6.6745257]
 [6.8128843]
 [6.641811 ]
 [6.16287  ]
 [6.399787 ]
 [6.604277 ]
 [6.1608906]] [[11.]
 [11.]
 [ 2.]
 [ 9.]
 [ 1.]
 [ 7.]
 [ 9.]
 [ 9.]
 [ 5.]
 [ 3.]
 [ 9.]
 [ 7.]
 [ 2.]
 [ 4.]]
10 15 LT
Epoch 1/100
110/110 [==============================] - 34s 308ms/step - loss: 0.0941
Epoch 2/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0497
Epoch 3/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0492
Epoch 4/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0488
Epoch 5/100
110/110 [==================

In [20]:
team="SS"
pitche=pitcher16[pitcher16.T_ID==team]

In [26]:
print(loss_min, box_min,look_back_min)

0.3981184744223738 10 5
